If captioning doesn't work, find similar looking person/people and take a top response from one of them

### Imports

In [1]:
import json
import requests
import pandas as pd
from tqdm import tqdm

### Constants

All comment fields:\
'name', 'edited', 'score', 'id', 'retrieved_on', 'score_hidden',  'parent_id', 'author_flair_css_class', 'gilded', 'link_id', 'author',  'downs', 'subreddit_id', 'distinguished', 'created_utc', 'body '   'subreddit', 'controversiality', 'removal_reason', 'author_flair_text    'ups', 'archived', 'stickied', 'author_cakeday', 'can_gil     'approved_at_utc', 'can_mod_post', 'banned_at_utc', 'collaps      'collapsed_reason', 'is_submitter', 'permalink', 'subreddit_t       'mod_note', 'mod_reason_by', 'mod_reason_title', 'no_fo
       'send_replies', 'author_flair_template_id', 'approved_by', 'bann
       'body_html', 'likes', 'mod_reports', 'num_reports', 'r,
       'report_reasons', 'saved', 'user_',
       'author_flair_background_color', 'author_flair_t',
     a 'author_flair_text_color', 'author_flair_type', de',
   r   'author_created_utc', 'author_fullname', 'subreddit_namxed',
       'gildings', 'author_patreon_flair', 'quarantinecked',
       'all_awardings', 'total_awards_received', 'steward_reportsrders',
       'a/

All submission fields:\
'is_self', 'from', 'author', 'link_flair_css_class', 'name', 'gilded', 'id', 'archived', 'author_flair_css_class',  'num_comments', 'saved', 'over_18', 'edited', 'score'   'quarantine', 'stickied', 'from_id', 'secure_media', 'url    'subreddit_id', 'from_kind', 'created', 'titl     'secure_media_embed', 'downs', 'thumbnail', 'distinguish      'domain', 'hide_score', 'created_utc', 'author_flair_t       'media_embed', 'subreddit', 'ups', 'retrieved_on', 'self
       'media', 'link_flair_text', 'permalink', 'preview', 'post
       'locked', 'contest_mode', 'spoiler', 'hidden', 'suggeste,
       'brand_safe', 'author_cakeday', 'view_count', 'c',
       'thumbnail_width', 'thumbnail_height', 'o',
       'approved_at_utc', 'banned_at_utc', 'canst',
       'is_crosspostable', 'num_crossposts', 'parent_whiteltus',
       'whitelist_status', 'is_reddit_media_domainned',
       'subreddit_type', 'mod_note', 'mod_reason_by', 'mod_title',
       'no_follow', 'send_replies', 'subreddiribers',
       'approved_by', 'author_flair_bad_color',
       'author_flair_template_id', 'author_flair_text_colanned_by',
       'category', 'clicked', 'content_categories', 'is_l_content',
       'likes', 'link_flair_richtext', 'linktext_color',
       'link_flair_type', 'media_only', 'mod_reponum_reports',
       'post_categories', 'pwls', 'removal_reasoport_reasons',
       'rte_mode', 'selftext_html', 'user_reporisited', 'wls',
       'author_flair_richtexthor_flair_type',
       'link_flair_background_colrosspost_parent',
       'crosspost_parent_list', lair_template_id',
       'previous_visits', 'media_metadauthor_created_utc',
       'author_fullname', 'is_meta',ddit_name_prefixed',
       'gildings', 'is_robot_indexabuthor_patreon_flair',
       'all_awardings', 'total_awards_rece'allow_live_comments',
       'discussion_type', 'awarders', 'remov 'removed_by_category',
       'author_premium', 'tt_tags', 'upvote_ratio',
       'is_created_from_ads_ui', 'retutc', 'top_awarded_type',
       'url_overridden_by_desent_end', 'event_is_live',
       'event_start', 'gallery_data', 'is_gallery', 'call_to_action'ociated_award'],

In [2]:
SUBMISSION_DATA = "../data/RoastMe_submissions"
COMMENTS_DATA = "../data/RoastMe_comments"
COMMENT_THRESHOLD = 500
COMMNET_LIMIT = 10000
SUBMISSION_FIELDS = [
    "name",
    "author",
    "title",
    "created_utc",
    "score",
    "is_self",
    "over_18",
    "selftext",
    "is_video",
    "is_reddit_media_domain",
    "is_gallery",
    "domain",
    "url"
]
COMMENT_FIELDS = [
    "id",
    "author",
    "created_utc",
    "parent_id", # filter on t3
    "link_id",
    "score",
    "controversiality",
    "is_submitter", # filter out true
    "body"
]
IMAGE_DOMAINS = [
    "i.redd.it",
    "imgur.com",
    "i.reddituploads.com",
    "i.imgur.com",
    "m.imgur.com",
]
IMAGE_CONTENT_TPYES = ["image/gif", "image/jpeg", "image/png"]  
REDDIT_DOIMAINS = ["i.redd.it", "i.reddituploads.com"]
tqdm.pandas()

### Functions

In [3]:
def lazy_load(file, stop=None):
    i = -1
    while True:
        data = file.readline()
        i += 1
        if not data or i == stop: break
        yield json.loads(data)

def is_top_level_comment(comment):
    return comment["parent_id"][:2] == "t3"

def get_top_comments(comments, n, threshold, submissions):
    i = 0
    top_comments = []
    with tqdm(total=n) as progress:
        for c in comments:
            if c["score"] >= threshold and is_top_level_comment(c):

                # Get parent post data
                post_data = get_parent_post_data(submissions, c["parent_id"])
                if not post_data:
                    continue

                # Skip if comment is OP
                if post_data["post_author"] == c["author"]:
                    continue
                    
                top_comments.append(c)
                i += 1
                progress.update(1)
                if i == n: break

    top_comments = pd.DataFrame(top_comments)[COMMENT_FIELDS]
    
    return top_comments

def get_parent_post_data(submissions, parent_id):
    post = submissions[submissions.name == parent_id]
    if len(post) == 0:
        return False

    post = post.iloc[0]
    post = post.add_prefix("post_")
    
    return post.to_dict()

# def get_content_type(domain, url):
#     if domain == "imgur.com":
#         url.replace("http://", "https://i.")
#         url.replace("Http://", "https://i.")
#         url.replace("https://", "https://i.")
#         url.replace("Https://", "https://i.")
#         url.replace("http://www.", "https://i.")
#         url.replace("Http://www.", "https://i.")
#         url.replace("https://www.", "https://i.")
#         url.replace("Https://www.", "https://i.")
        
#     try:
#         header = requests.head(url)
#     except Exception as e:
#         print(e)
#         return "failed"
#     try:
#         return header.headers["content-type"]
#     except:
#         return header.status_code

def fix_imgur_url(domain, url):
    if domain == "imgur.com":
        url = url.replace("https://", "https://i.")
        url = url.replace("http://", "https://i.")
        url = url.replace("Http://", "https://i.")
        url = url.replace("Https://", "https://i.")
        url = url.replace("http://www.", "https://i.")
        url = url.replace("Http://www.", "https://i.")
        url = url.replace("https://www.", "https://i.")
        url = url.replace("Https://www.", "https://i.")
        if url[-3:] not in ["jpg", "gif", "png" "peg"]:
            url += ".jpg"
        
    return url

def filter_and_format_submissions(submissions):
    #remove self posts
    submissions = submissions[~submissions.is_self]
    # remove non image/obscure image hosted posts
    submissions = submissions[submissions.domain.isin(IMAGE_DOMAINS)]
    # remove deleted image posts
    submissions = submissions[~((submissions.selftext == "[deleted]") & (submissions.domain.isin(REDDIT_DOIMAINS)))]
    # remove removed posts
    submissions = submissions[submissions.selftext != "[removed]"]
    #remove gallery posts
    submissions = submissions[submissions.is_gallery != True]
    # merge url fields
    submissions["url"] = submissions.apply(lambda post: post.url if post.url else post.url_overridden_by_dest, axis=1)
    # fix imgur urls
    submissions["url"] = submissions.apply(lambda post: fix_imgur_url(post.domain, post.url), axis=1)

    return submissions[SUBMISSION_FIELDS]

### Load Data
16136317 total comments

439967 total posts

#### Load Submissions

In [4]:
with open(SUBMISSION_DATA, "r", encoding='cp850') as f:
    submissions = pd.DataFrame([json.loads(line) for line in f.readlines()])
print(submissions.shape)
submissions.head()

(439967, 129)


,is_self,from,author,link_flair_css_class,name,gilded,id,archived,author_flair_css_class,num_comments,...,is_created_from_ads_ui,retrieved_utc,top_awarded_type,url_overridden_by_dest,event_end,event_is_live,event_start,gallery_data,is_gallery,call_to_action
0,False,NaN,[deleted],None,t3_33fw7y,0,33fw7y,False,None,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,True,NaN,[deleted],None,t3_33fwxm,0,33fwxm,False,None,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,True,NaN,[deleted],None,t3_33fwyn,0,33fwyn,False,None,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,False,NaN,[deleted],,t3_33fxn6,0,33fxn6,False,None,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,True,NaN,iwillkicku67,,t3_33fytd,0,33fytd,False,ham,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Filter Submissions

In [5]:
submissions = filter_and_format_submissions(submissions)
print(submissions.shape)
submissions.head()

(318498, 13)


,name,author,title,created_utc,score,is_self,over_18,selftext,is_video,is_reddit_media_domain,is_gallery,domain,url
3,t3_33fxn6,[deleted],Do your worst!,1429678694,2,False,True,,NaN,NaN,NaN,imgur.com,https://i.imgur.com/WuVB0FT.jpg
9,t3_33iz0o,SwagmasterEDP,do your worst!,1429740688,38,False,False,,NaN,NaN,NaN,imgur.com,https://i.imgur.com/lg4GoAs.jpg
10,t3_33j2l2,[deleted],/u/Tyoko,1429742238,3,False,False,,NaN,NaN,NaN,imgur.com,https://i.imgur.com/VJ2TVZC.jpg
11,t3_3bti9n,[deleted],Not saying if I'm the fish or the guy.,1435798027,209,False,False,,NaN,NaN,NaN,i.imgur.com,http://i.imgur.com/zkjSV0D.jpg
12,t3_3btkg6,Tyrosoldier,roast me you betafucks,1435799165,16,False,False,,NaN,NaN,NaN,i.imgur.com,http://i.imgur.com/bbqZD0B.jpg


#### Load Comments

In [6]:
with open(COMMENTS_DATA, "r", encoding='cp850') as f:
    comments = lazy_load(f)
    top_comments = get_top_comments(comments, COMMENT_LIMIT, COMMENT_THRESHOLD, submissions)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [09:49<00:00, 16.95it/s]


In [7]:
top_comments.head()

,id,author,created_utc,parent_id,link_id,controversiality,is_submitter,body
0,cspcsxv,Tyrosoldier,1435798221,t3_3bti9n,t3_3bti9n,0,NaN,dis nigga look like the penguin
1,cspe2vi,SwagmasterEDP,1435800586,t3_3btn16,t3_3btn16,0,NaN,I think your barber already did.
2,cspewzn,annieareyouokayannie,1435802162,t3_3btmws,t3_3btmws,0,NaN,You look like the kind of guy I would have mad...
3,cspfv6y,sexual_pancakes,1435804002,t3_3btn16,t3_3btn16,0,NaN,Why you look like a cheap backstreet boy? \n\n...
4,cspgter,Bears54,1435805849,t3_3btvzx,t3_3btvzx,0,NaN,You doing the same expression her dad made whe...


### Save Data

In [9]:
top_comments.to_csv("../data/top_comments_10000_500.csv", index=False)
submissions.to_csv("../data/submissions.csv", index=False)